In [66]:
%reset -f 
Station="IntD"
# This script is used to generate a grid search for the IntD station
# name file Station+grid_search.py
name_file = Station + "_grid_search.py"
with open(name_file, "w") as f:
    pass

In [67]:
# -*- coding: utf-8 -*-

"""
Created on Wed Oct 18 14:57:23 2023
import functions mesh
"""
dir_ginette = "/home/ariviere/Programmes/ginette"
# RIV2D + Station

import os
main_dir = os.path.join(dir_ginette, "application/RIV2D", Station)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import math
import matplotlib.pyplot as plt
# Ajouter le chemin du dossier src au PYTHONPATH
import importlib
sys.path.append(dir_ginette)  # Ajouter le dossier parent de src

# use __init__.py to import all functions/ in src/src_gmsh and src/src_pythont
# insure 
from src.src_gmsh import mesh_generator
from src.src_python import Init_folders
from src.src_python import Direct_model
from src.src_python import Read_obs
from src.src_python import Plot
from src.src_python import stat_critere
from src.src_python import Grid_search

# Import all functions/classes from the relevant modules
from src.src_gmsh.mesh_generator import *
from src.src_python.Init_folders import *
from src.src_python.Direct_model import *
from src.src_python.Read_obs import *
from src.src_python.Plot import *
from src.src_python.stat_critere import *
from src.src_python.Grid_search import *
import importlib


importlib.reload(Init_folders)
importlib.reload(Direct_model)
importlib.reload(Grid_search)

# give me all names of the functions in Init_folders
#print(Init_folders.__dict__)
#print([name for name in dir(Init_folders) if callable(getattr(Init_folders, name)) and not name.startswith("__")])



<module 'src.src_python.Grid_search' from '/home/ariviere/Programmes/ginette/src/src_python/Grid_search.py'>

# PARAMETERS RANGE
You should already have your table ready
Remove the ginette file in the Station repertory to be sure to recompile

In [68]:


# Change the working directory to the main simulation directory
os.chdir(main_dir)

# Compile the Ginette source code (if needed for the simulation)
compile_ginette_src(dir_ginette)


ginette exists


In [69]:
# remove IntD_grid_search.py
name_file = Station + "_grid_search.py"
os.remove(name_file)

In [70]:
%%writefile  IntD_grid_search.py

import os
import sys
# read the input_inversion.txt file#%%writefile IntD_grid_search.py
with open("input_inversion.txt", "r") as f:
    lines = f.readlines()

# Helper to parse a line of the form "key: value"
def parse_line(line):
    key, value = line.strip().split(":", 1)
    return key.strip(), value.strip()

parsed = dict(parse_line(line) for line in lines)

date_simul_bg = parsed["date_simul_bg"]
Station = parsed["Station"]
# Read param_struct from the parsed input and convert to list
param_struct = [x.strip(" '") for x in parsed["param_struct"].split(",")]
sensors = [x.strip(" '") for x in parsed["sensors"].split(",")]
sigma = float(parsed["sigma"])
zones_to_invert = [int(z) for z in parsed["zones_to_invert"].strip("[]").split(",")]
parameters_to_invert = [x.strip(" '") for x in parsed["parameters_to_invert"].split(",")]
simul_todo_range = parsed["simul_todo"].split(",")
simul_todo = range(int(simul_todo_range[0]), int(simul_todo_range[1]))
main_dir = parsed["path_simul"]
dir_ginette = parsed["dir_ginette"]

os.chdir(main_dir)





import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import math
import matplotlib.pyplot as plt
# Ajouter le chemin du dossier src au PYTHONPATH
import importlib
sys.path.append(dir_ginette)  # Ajouter le dossier parent de src

# use __init__.py to import all functions/ in src/src_gmsh and src/src_pythont
# insure 
from src.src_gmsh import mesh_generator
from src.src_python import Init_folders
from src.src_python import Direct_model
from src.src_python import Read_obs
from src.src_python import Plot
from src.src_python import stat_critere
from src.src_python import Grid_search

# Import all functions/classes from the relevant modules
from src.src_gmsh.mesh_generator import *
from src.src_python.Init_folders import *
from src.src_python.Direct_model import *
from src.src_python.Read_obs import *
from src.src_python.Plot import *
from src.src_python.stat_critere import *
from src.src_python.Grid_search import *
import importlib


importlib.reload(Init_folders)
importlib.reload(Direct_model)
importlib.reload(Grid_search)
from src.src_python.Grid_search import *
from src.src_python import Grid_search
importlib.reload(Grid_search)

# Convert the date_simul_bg to datetime
date_simul_bg = pd.to_datetime(date_simul_bg, format='%Y-%m-%d %H:%M:%S')
print(f"date_simul_bg: {date_simul_bg}")
print(f"Station: {Station}")
print(f"param_struct: {param_struct}")
print(f"sensors: {sensors}")
print(f"sigma: {sigma}")
print(f"zones_to_invert: {zones_to_invert}")
print(f"parameters_to_invert: {parameters_to_invert}")
print(f"simul_todo: {simul_todo}")
print(f"path_simul: {main_dir}")
print(f"dir_ginette: {dir_ginette}")


# read the param_table.txt file
param_table = pd.read_csv("param_table.txt", sep=",", header=0)
param_table['index_sim'] = param_table.index + 1
header_save_param = param_table.columns

# Table of simulated parameters
# param_table = pd.DataFrame(columns=header_save_param)
param_table_simul = pd.DataFrame(columns=header_save_param)

# make a loop to run each index_sim in simul_todo

for id_sim in simul_todo:
    print(f"Simulation {id_sim}")
    param_table_id_sim = run_one_simulation_2D(
        Station, main_dir, date_simul_bg, sensors,
        param_table, id_sim, param_struct, zones_to_invert,
        parameters_to_invert, param_table_simul, dir_ginette)

    path_output_tempsim = os.path.join(main_dir, f"OUTPUT_{Station}")
    new_file_name = f"{id_sim}_S_flux_ZH_aq.dat"
    # save in the folder OUTPUT path_output_tempsim
    new_file_path = os.path.join(path_output_tempsim, new_file_name)
    os.rename("S_flux_ZH_aq.dat", new_file_path)
    new_file_name= os.path.join(f"{id_sim}_S_pression_charge_temperature.dat")
    # rename S_pression_charge_temperature.dat to new_file
    new_file_path = os.path.join(path_output_tempsim, new_file_name)
    # rename S_flux_ZH_aq.dat to new_file_path
    os.rename("S_pression_charge_temperature.dat",new_file_path)
    

param_table_simul = pd.concat([param_table_simul, param_table_id_sim], ignore_index=True)


param_table_simul.to_csv("PARAMETER_" + Station + "/"+ "S_parmeters_simul.dat", index=False)

Writing IntD_grid_search.py
